This is notebook to know what is the outcome of each file, what is the distribution of each running Hz.

In [1]:
import os
import sys
import re
import numpy as np

In [2]:
ABSOLUTE_DIR = '/home/phong/driving_data/original_gamma/'
#ABSOLUTE_DIR = '/home/phong/driving_data/original_gamma/'

In [3]:
ERROR_CODE = {
    'GOAL': 0,
    'COLLISION': 1,
    'LARGE_VEL': 2,
    'PATH_OFFSET': 3,
    'EGO_KILLED': 4,
    'EPS_LIMIT': 5,
    'PROCESS_KILL': 6,
    'BELIEF_NORM_0': 7,
    'INTENTION_ID': 8,
    'EGO_LESS_THAN_AGENT': 9,
    'STOP_SUDDENLY': 10,
}

In [4]:


def read_file_and_return_outcomes(file_name):
    eps_len = 120 # 120 seconds for time_scale = 1. 
    time_scale = None
    first_time_ego = None
    last_time_ego = None
    first_time_agent = None
    last_time_agent = None
    with open(file_name, 'r') as f:
        lines = f.readlines()
        for line in lines:
            if "time_scale=" in line:
                time_scale = float(line.split('e=')[1])
                eps_len = int(120 / time_scale) # This is the maximum time a episode can run

            if "get car state at" in line:
                current_time = float(line.split("t=")[1])
                if not first_time_ego:
                    first_time_ego = current_time
                last_time_ego = current_time

            if "agents at time" in line:
                current_time = float(line.split(",")[0].split(" ")[-1])
                if not first_time_agent:
                    first_time_agent = current_time
                last_time_agent = current_time
                
            if 'Unusual car vel (too large)' in line:
                #print("file {} has large vel".format(file_name))
                return ERROR_CODE['LARGE_VEL']
            if "goal reached" in line:
                return ERROR_CODE['GOAL']
            if "collision = 1" in line:
                return ERROR_CODE['COLLISION']
            if "After reaching goal: real_spead is already zero" in line:
                return ERROR_CODE['GOAL']
            if "Path offset too high !!! Node shutting down" in line:
                return ERROR_CODE['PATH_OFFSET']
            if "Ego vehicle killed in ego_vehicle.py" in line:
                return ERROR_CODE['EGO_KILLED']
            if "total_prob == 0" in line:
                return ERROR_CODE['BELIEF_NORM_0']
            if "ERROR: Intention ID" in line:
                return ERROR_CODE['INTENTION_ID']
            
    # If error, both ego and agent time should be small
    if (first_time_agent == None and first_time_ego == None) or \
        ((last_time_ego - first_time_ego < 0.15*eps_len) and (last_time_agent - first_time_agent < 0.15*eps_len)):
        return ERROR_CODE['PROCESS_KILL']
    
    # If episode exceeds limits, both ego and agent time should be exceed
    if (last_time_ego - first_time_ego > 0.95*eps_len) and (last_time_agent - first_time_agent > 0.95*eps_len):
        return ERROR_CODE['EPS_LIMIT']
    
    # The left case would be go < 0.95 but agent > 0.95. THis is error because no callback from ego vehicle
    if (last_time_ego - first_time_ego < 0.95*eps_len) and (last_time_agent - first_time_agent > 0.95*eps_len):
        return ERROR_CODE['EGO_LESS_THAN_AGENT']
    # Assertion here because the error
    
    return ERROR_CODE['STOP_SUDDENLY']

    assert False, f"File {file_name} does not have desired outcome {last_time_ego - first_time_ego} {last_time_agent - first_time_agent}"

def get_outcomes_from_dir(dir_name):
    outcomes = [0 for _ in range(len(ERROR_CODE))]

    for root, subdirs, files in os.walk(ABSOLUTE_DIR):
        if len(files) > 0:
            for file in files:
                if file.endswith('.txt'):
                    file_path = os.path.join(root, file)
                    outcome = read_file_and_return_outcomes(file_path)
                    outcomes[outcome] += 1

In [5]:
outcomes = [0 for _ in range(len(ERROR_CODE))]

for root, subdirs, files in os.walk(ABSOLUTE_DIR):
    if len(files) > 0:
        for file in files:
            if file.endswith('.txt'):
                file_path = os.path.join(root, file)
                outcome = read_file_and_return_outcomes(file_path)
                outcomes[outcome] += 1


In [7]:
# print outcomes which is number of files for each category:
# Convert ERROR_CODE to dict with key is number 
ERROR_CODE = {v: k for k, v in ERROR_CODE.items()}
for i in range(len(outcomes)):
    print(f"{ERROR_CODE[i]}: {outcomes[i]}")

GOAL: 0
COLLISION: 60
LARGE_VEL: 0
PATH_OFFSET: 0
EGO_KILLED: 240
EPS_LIMIT: 207
PROCESS_KILL: 60
BELIEF_NORM_0: 6
INTENTION_ID: 1
EGO_LESS_THAN_AGENT: 4
STOP_SUDDENLY: 1


In this part, I do simple statistics of tree:
1/ Depth
2/ Number of expanded nodes
3/ Total nodes

4/ Rewards

In [2]:
def read_file_and_return_performances(file_name):
    depths = []
    trials = []
    expanded_nodes = []
    total_nodes = []
    rewards = []

    with open(file_name, 'r') as f:
        lines = f.readlines()
        for line in lines:
            if "# nodes: expanded" in line:
                expanded, total, policy = line.split("=")[-1].split("/")
                expanded_nodes.append(int(expanded))
                total_nodes.append(int(total))
            if "Trials: no." in line:
                depth = int(line.split("=")[1].split("/")[-1])
                trial = int(line.split("=")[1].split("/")[0])
                depths.append(depth)
                trials.append(trial)

            if "reward **=" in line:
                rewards.append(float(line.split("=")[-1]))

    return depths, trials, expanded_nodes, total_nodes, rewards

In [3]:

ABSOLUTE_DIR = '/home/phong/driving_data/original_gamma/'
mean_depths = []
mean_trials = []
mean_expanded_nodes = []
mean_total_nodes = []
mean_rewards = []
for root, subdirs, files in os.walk(ABSOLUTE_DIR):
    if len(files) > 0:
        for file in files:
            if file.endswith('.txt'):
                #print(f"Reading file {root}/{file}")
                file_path = os.path.join(root, file)
                depths, trials, expanded_nodes, total_nodes, rewards = read_file_and_return_performances(file_path)
                if len(depths) == 0:
                    continue
                mean_depths.append(np.mean(depths))
                mean_trials.append(np.mean(trials))
                mean_expanded_nodes.append(np.mean(expanded_nodes))
                mean_total_nodes.append(np.mean(total_nodes))
                mean_rewards.append(np.mean(rewards))

                # print(f"depths: {np.mean(depths)}")
                # print(f"trials: {np.mean(trials)}")
                # print(f"expanded_nodes: {np.mean(expanded_nodes)}")
                # print(f"total_nodes: {np.mean(total_nodes)}")
                # print(f"rewards: {np.mean(rewards)}")
            #break
# print all the mean
print(f"mean_depths: {np.mean(mean_depths)}")
print(f"mean_trials: {np.mean(mean_trials)}")
print(f"mean_expanded_nodes: {np.mean(mean_expanded_nodes)}")
print(f"mean_total_nodes: {np.mean(mean_total_nodes)}")
print(f"mean_rewards: {np.mean(mean_rewards)}")


mean_depths: 7.521030030139836
mean_trials: 464.6465879213932
mean_expanded_nodes: 759.6961692787397
mean_total_nodes: 2138.4852491520533
mean_rewards: -2.235645391309308


In [16]:
ABSOLUTE_DIR = '/home/phong/driving_data/gamma_1threads/'
mean_depths = []
mean_trials = []
mean_expanded_nodes = []
mean_total_nodes = []
mean_rewards = []
for root, subdirs, files in os.walk(ABSOLUTE_DIR):
    if len(files) > 0:
        for file in files:
            if file.endswith('.txt'):
                #print(f"Reading file {root}/{file}")
                file_path = os.path.join(root, file)
                depths, trials, expanded_nodes, total_nodes, rewards = read_file_and_return_performances(file_path)
                if len(depths) == 0:
                    continue
                mean_depths.append(np.mean(depths))
                mean_trials.append(np.mean(trials))
                mean_expanded_nodes.append(np.mean(expanded_nodes))
                mean_total_nodes.append(np.mean(total_nodes))
                mean_rewards.append(np.mean(rewards))

                # print(f"depths: {np.mean(depths)}")
                # print(f"trials: {np.mean(trials)}")
                # print(f"expanded_nodes: {np.mean(expanded_nodes)}")
                # print(f"total_nodes: {np.mean(total_nodes)}")
                # print(f"rewards: {np.mean(rewards)}")
            #break
# print all the mean
print(f"mean_depths: {np.mean(mean_depths)}")
print(f"mean_trials: {np.mean(mean_trials)}")
print(f"mean_expanded_nodes: {np.mean(mean_expanded_nodes)}")
print(f"mean_total_nodes: {np.mean(mean_total_nodes)}")
print(f"mean_rewards: {np.mean(mean_rewards)}")


mean_depths: 6.092680190882702
mean_trials: 20.356812577105764
mean_expanded_nodes: 43.16904586682518
mean_total_nodes: 111.25153285782861
mean_rewards: -2.396472451905705


In [18]:
ABSOLUTE_DIR = '/home/phong/driving_data/lanegcn0_1_test14/'
mean_depths = []
mean_trials = []
mean_expanded_nodes = []
mean_total_nodes = []
mean_rewards = []
for root, subdirs, files in os.walk(ABSOLUTE_DIR):
    if len(files) > 0:
        for file in files:
            if file.endswith('.txt'):
                #print(f"Reading file {root}/{file}")
                file_path = os.path.join(root, file)
                depths, trials, expanded_nodes, total_nodes, rewards = read_file_and_return_performances(file_path)
                if len(depths) == 0:
                    continue
                mean_depths.append(np.mean(depths))
                mean_trials.append(np.mean(trials))
                mean_expanded_nodes.append(np.mean(expanded_nodes))
                mean_total_nodes.append(np.mean(total_nodes))
                mean_rewards.append(np.mean(rewards))

                # print(f"depths: {np.mean(depths)}")
                # print(f"trials: {np.mean(trials)}")
                # print(f"expanded_nodes: {np.mean(expanded_nodes)}")
                # print(f"total_nodes: {np.mean(total_nodes)}")
                # print(f"rewards: {np.mean(rewards)}")
            #break
# print all the mean
print(f"mean_depths: {np.mean(mean_depths)}")
print(f"mean_trials: {np.mean(mean_trials)}")
print(f"mean_expanded_nodes: {np.mean(mean_expanded_nodes)}")
print(f"mean_total_nodes: {np.mean(mean_total_nodes)}")
print(f"mean_rewards: {np.mean(mean_rewards)}")


mean_depths: 4.8582280972087775
mean_trials: 10.815689355182316
mean_expanded_nodes: 21.265743568103478
mean_total_nodes: 49.42439164767002
mean_rewards: -2.2629980048776317


In [4]:

ABSOLUTE_DIR = '/home/phong/driving_data/same_Hz/hivt3hz_fixed'
mean_depths = []
mean_trials = []
mean_expanded_nodes = []
mean_total_nodes = []
mean_rewards = []
for root, subdirs, files in os.walk(ABSOLUTE_DIR):
    if len(files) > 0:
        for file in files:
            if file.endswith('.txt'):
                #print(f"Reading file {root}/{file}")
                file_path = os.path.join(root, file)
                depths, trials, expanded_nodes, total_nodes, rewards = read_file_and_return_performances(file_path)
                if len(depths) == 0:
                    continue
                mean_depths.append(np.mean(depths))
                mean_trials.append(np.mean(trials))
                mean_expanded_nodes.append(np.mean(expanded_nodes))
                mean_total_nodes.append(np.mean(total_nodes))
                mean_rewards.append(np.mean(rewards))

                # print(f"depths: {np.mean(depths)}")
                # print(f"trials: {np.mean(trials)}")
                # print(f"expanded_nodes: {np.mean(expanded_nodes)}")
                # print(f"total_nodes: {np.mean(total_nodes)}")
                # print(f"rewards: {np.mean(rewards)}")
            #break
# print all the mean
print(f"mean_depths: {np.mean(mean_depths)}")
print(f"mean_trials: {np.mean(mean_trials)}")
print(f"mean_expanded_nodes: {np.mean(mean_expanded_nodes)}")
print(f"mean_total_nodes: {np.mean(mean_total_nodes)}")
print(f"mean_rewards: {np.mean(mean_rewards)}")


mean_depths: 0.12117889972108209
mean_trials: 1.4337029211386783
mean_expanded_nodes: 1.6702443862608376
mean_total_nodes: 14.120152768435966
mean_rewards: -2.7486576331937456
